In [0]:
host = dbutils.secrets.get(scope = "postgresql", key = "host")
database = dbutils.secrets.get(scope = "postgresql", key = "database")
user = dbutils.secrets.get(scope = "postgresql", key = "user")
password = dbutils.secrets.get(scope = "postgresql", key = "password")

In [0]:
financial_reports_table = (spark.read
  .format("postgresql")
  .option("dbtable", "public.financial_reports_raw")
  .option("host", host)
  .option("port", "5432") 
  .option("database", database)
  .option("user", user) 
  .option("password", password)
  .load()
)

In [0]:
display(financial_reports_table)

In [0]:
%python
display(financial_reports_table.dtypes)

In [0]:
%python
from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, "plstocks.bronze_financial_reports"):
    deltaTable = DeltaTable.forPath(spark, "plstocks.bronze_financial_reports")
    
    deltaTable.alias("existing") \
        .merge(
            financial_reports_table.alias("new"),
            "existing.ticker = new.ticker AND existing.year = new.year"
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    financial_reports_table.write.format("delta").mode("overwrite").saveAsTable("plstocks.bronze_financial_reports")